In [1]:
from MIOFlow.models import make_model
import torch

In [2]:
batch_size = 4
feature_dim = 5
output_dim = 5

x = torch.randn(batch_size, feature_dim)
# m0 = torch.ones(batch_size, 1)
# xm = torch.cat([x, m0], dim=1)
t = torch.linspace(0, 1, 7)

model = make_model(feature_dims=feature_dim, which='ode_growth_rate')

In [3]:
xt, mt = model(x, t)

In [4]:
xt.shape, mt.shape

(torch.Size([4, 5]), torch.Size([4]))

In [5]:
xtseq, mtseq = model(x, t, return_whole_sequence=True)

In [6]:
xtseq.shape

torch.Size([7, 4, 5])

In [7]:
mtseq.shape

torch.Size([7, 4])

In [8]:
torch.nn.functional.softmax(mt, dim=-1) * mt.shape[-1]

tensor([0.9834, 1.2876, 0.8258, 0.9032], grad_fn=<MulBackward0>)

In [9]:
torch.exp(mt)/torch.exp(mt).sum(dim=0)

tensor([0.2459, 0.3219, 0.2065, 0.2258], grad_fn=<DivBackward0>)

In [10]:
torch.nn.functional.softmax(mtseq, dim=-1) * mtseq.shape[-1]

tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [0.9969, 1.0418, 0.9758, 0.9855],
        [0.9937, 1.0866, 0.9498, 0.9700],
        [0.9909, 1.1338, 0.9217, 0.9535],
        [0.9884, 1.1828, 0.8917, 0.9370],
        [0.9860, 1.2340, 0.8597, 0.9203],
        [0.9834, 1.2876, 0.8258, 0.9032]], grad_fn=<MulBackward0>)

In [11]:
xtseq.shape

torch.Size([7, 4, 5])

In [12]:
mtseq.unsqueeze(2).shape

torch.Size([7, 4, 1])

In [13]:
xmseq = torch.cat([xtseq, mtseq.unsqueeze(2)], dim=-1)
xmseq.shape

torch.Size([7, 4, 6])

In [14]:
dxdts = torch.stack([model.func(t[i], xmseq[i]) for i in range(len(t))])

In [15]:
dxdts.shape

torch.Size([7, 4, 6])

In [16]:
mt.unsqueeze(1).shape

torch.Size([4, 1])

In [17]:
model.func

ToyODE(
  (seq): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [18]:
xt.shape

torch.Size([4, 5])

In [19]:
mt.shape

torch.Size([4])

In [20]:
xm = torch.cat([xt, mt.unsqueeze(-1)], dim=-1)
dxdt = model.func(t[-1], xm)

In [21]:
dxdt[...,:-1].shape

torch.Size([4, 5])

In [22]:
mtseq.shape

torch.Size([7, 4])

In [23]:
xtseq.shape

torch.Size([7, 4, 5])

In [24]:
t.shape

torch.Size([7])

In [25]:
t[[0,1,2],...]

tensor([0.0000, 0.1667, 0.3333])

In [26]:
mt.shape

torch.Size([4])

In [27]:
(dxdt * mt.unsqueeze(-1)).shape

torch.Size([4, 6])

In [28]:
dxdts.shape

torch.Size([7, 4, 6])

In [29]:
mtseq.shape

torch.Size([7, 4])

In [30]:
(dxdts * mtseq.unsqueeze(-1)).shape

torch.Size([7, 4, 6])

In [31]:
torch.nn.functional.softmax(mtseq, dim=-1).shape

torch.Size([7, 4])

In [32]:
mtseq.shape[-1]

4

In [33]:
torch.nn.functional.softmax(mt, dim=-1).shape

torch.Size([4])

In [34]:
mt.shape[-1]

4

In [35]:
def func_end_0(t, x):
    xm = model.func(t, x)
# mt[[0,2,4,6,8]] = 0.
model.func(t[-1], xm).shape

torch.Size([4, 6])

In [36]:
xm[...,-1].shape

torch.Size([4])

In [37]:
xm.shape

torch.Size([4, 6])

In [38]:
xmseq.shape

torch.Size([7, 4, 6])

In [39]:
xm[[0,2],-1] = 0.

In [40]:
xm

tensor([[-2.4157,  0.2534,  0.4717,  0.6886,  1.0636,  0.0000],
        [-1.0166, -0.3361,  3.1126,  0.2458,  1.5748,  0.2797],
        [-0.2218,  0.0036, -1.1669,  1.7149, -1.3935,  0.0000],
        [ 0.1466, -0.1166, -1.1889,  2.0777,  0.8608, -0.0749]],
       grad_fn=<CopySlices>)

In [41]:
dxmdt = model.func(t[-1], xm)

In [42]:
dxmdt

tensor([[ 0.1817, -0.2363,  0.0289,  0.0854,  0.0905,  0.0508],
        [ 0.2975, -0.2181,  0.4918, -0.0032, -0.5188,  0.3222],
        [ 0.1845,  0.3455, -0.2675, -0.2054,  0.0796, -0.1937],
        [ 0.2735,  0.0493,  0.0954, -0.0474, -0.0049, -0.0480]],
       grad_fn=<AddmmBackward0>)